# Trabalho Final - Data Mining II - IESB

Esse projeto foi realizado como entrega final da matéria de Data Mining II da Pós Graduação de Ciência de Dados no IESB.
Para execução do projeto foi realizada a competição Costa Rican Household Poverty Level Prediction - https://www.kaggle.com/c/costa-rican-household-poverty-prediction
    
Como base do projeto foi utilizado o notebook disponibilizado pelo professor Marcos https://www.kaggle.com/marcosvafg/iesb-miner-ii-aula-05-random-forest e o objetivo era ter uma submissão maior do que 0.43719.

Foi obtido um valor de 0.44109 na competição, atigindo o objetivo primário da matéria.

# Importações iniciais


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
# Carregando os dados do desafio
train = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

print('Tamanho dos datasets Train e Test:')
print(' - Train - Linhas:', train.shape[0],'Colunas:', train.shape[1],'\n - Test - Linhas:', test.shape[0],'Colunas:', test.shape[1])

In [ ]:
# Juntando os dataframes - train e test
df_all = train.append(test)

print('Informações do novo dataset:\n df_all - Linhas:', df_all.shape[0],'Colunas:', df_all.shape[1])

# Tratamento dos dados

Passo 1. Verificação do dataset para validar se os dados inseridos são os dados esperados pelo dataset. Caso seja esperado dado númerico e o campo for object, esses registros precisam ser tratados.

In [ ]:
# Quais colunas do dataframe são do tipo object
# Importante passo para saber se alguma coluna tem dados (linhas) 
#    com tipo diferente
df_all.select_dtypes('object').head()



Como pode ser visualizado na saída acima, as colunas dependency, edjefa e edjefe são do tipo object. Nas instruções do desafio informava o seguinte:

* dependency, Dependency rate, calculated = (number of members of the household younger than 19 or older than 64)/(number of member of household between 19 and 64)
* edjefe, years of education of male head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0
* edjefa, years of education of female head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0

Ou seja, valores númericos, então nesse caso precisam ser tratados.


In [ ]:
# Analisando os dados da coluna edjefa
df_all['edjefa'].value_counts()

In [ ]:
# Analisando os dados da coluna edjefe
df_all['edjefe'].value_counts()

In [ ]:
# Analisando os dados da coluna dependency
df_all['dependency'].value_counts()

Após análise dos dados das colunas edjefa, edjefe e dependency, temos:

Para a coluna edjefa, temos 214: yes e 22075: no

Para a coluna edjefe, temos 416: yes e 12818: no

Para a coluna dependecy, temos 7580: yes e 6036: no


Para alteração dos registros, será utilizada a função abaixo, onde:
* yes será 1
* no será 0

In [ ]:
# Função para excecutar troca dos valores yes e no, para 1 e 0, respectivamente
mapeamento = {'yes': 1, 'no': 0}

In [ ]:
# Transformação dos dados das colunas edjefa e edjefe
df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

Após execução, espera que apenas a coluna dependency ainda seja object

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Olhando a coluna dependency
df_all['dependency'].value_counts()

Com a execução anterior podemos verificar que as variáveis edjefa e edjefe não são mais do tipo object, agora apenas coluna dependecy possui dados desse tipo yes/no e precisam ser tratados.

Como a variável dependecy possui uma variável SQBdependency com os dados 'Ao Quadrado', iremos tratá-la com os dados da raiz quadrada de SQBdependency

In [ ]:
# Verificando as colunas dependency e SQBdependency, pois uma é baseada na outra.
df_all[['dependency','SQBdependency']]

In [ ]:
# Preenchimento dos valores de dependency 
#  com os valores da raiz quadrada de SQBdependency
df_all['dependency'] = np.sqrt(df_all['SQBdependency'])

Agora não é esperado mais colunas do tipo object.

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

Após tratamento dos registros das colunas objects, vamos verificar se temos valores nulos no dataset

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum().sort_values()

Temos algumas colunas como dados nulos, vamos tratar primeiramente as que mais tem dados, pois saída acima está ordenada. São elas: v2a1, v18q1 e rez_esc.

In [ ]:
# Tratamento dos valores nulos nas variáveis: v2a1, v18q1 e rez_esc
#df_all['v2a1'].fillna(0, inplace=True)
#df_all['v18q1'].fillna(0, inplace=True)
#df_all['rez_esc'].fillna(0, inplace=True)
df_all['v2a1'] = df_all['v2a1'].fillna(value=df_all['tipovivi3'])
df_all['v18q1'] = df_all['v18q1'].fillna(value=df_all['v18q'])
df_all['rez_esc'].fillna(0, inplace=True)

Registros tratados, vamos tratar os outros dados que ainda possuem nulo

In [ ]:
# Verificando os valores nulos restantes
df_all.isnull().sum().sort_values()

Como temos poucos dados e iremos atribuir 0 para as colunas meaneduc e SQBmeaned

In [ ]:
# Prenchendo com 0 todos os valores nulos
df_all['meaneduc'].fillna(0, inplace=True)
df_all['SQBmeaned'].fillna(0, inplace=True)

In [ ]:
df_all['Target'].fillna(-1, inplace=True)

# Bibliotecas

In [ ]:
# Carregando as bibliotecas utilizadas para os modelos

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, make_scorer

# Modelo 1

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Separar os dataframes
train, test = df_all[df_all['Target'] != -1], df_all[df_all['Target'] == -1]

In [ ]:
# Instanciando RadomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, n_estimators=200, oob_score=True, random_state=42)

In [ ]:
# Treinando o modelo
rf.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
# Vamos verificar as previsões
test['Target'].value_counts(normalize=True)

In [ ]:
# Criando o arquivo para submissão
#test[['Id', 'Target']].to_csv('submission.csv', index=False)

# Modelo 2



Após primeira submissão após limpeza da base. Tivemos um resultado de 0.43777. Agora iremos fazer novas limpezas para verificar se o valor pode ser melhorado.


In [ ]:
# Feature Engineering

# Vamos criar novas colunas para valores percapita
df_all['hsize-pc'] = df_all['hhsize'] / df_all['tamviv']
df_all['phone-pc'] = df_all['qmobilephone'] / df_all['tamviv']
df_all['tablets-pc'] = df_all['v18q1'] / df_all['tamviv']
df_all['rooms-pc'] = df_all['rooms'] / df_all['tamviv']
df_all['rent-pc'] = df_all['v2a1'] / df_all['tamviv']

In [ ]:
# Separar os dataframes
train, test = df_all[df_all['Target'] != -1], df_all[df_all['Target'] == -1]


Para isso utilizaremos matrix de correlação e também verificação no dataset para variáveis que sejam basicamente a mesma coisa. O código abaixo foi retirado do kaggle https://www.kaggle.com/willkoehrsen/a-complete-introduction-and-walkthrough para verificar a correlação das variáveis. Alguns ajustes foram realizados.

Para realizar a correlação foram utilizados:

np.triu: Upper triangle of an array. Return a copy of a matrix with the elements below the k-th diagonal zeroed.

np.ones: Return a new array of given shape and type, filled with ones.

Para que a colune fosse considerada correlacionada e pudesse ser retirada, foi utlizado o parâmetro de 0.975, fazendo o usu do valor absoluto da coluna (abs).


In [ ]:
# Utilizando matriz de correspondência para verificar se algumas variáveis podem ser retiradas do modelo

# Create correlation matrix
corr = df_all.corr()

# Select upper triangle of correlation matrix
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.975
to_drop = [column for column in upper.columns if any(abs(upper[column]) > 0.975)]

print(f'There are {len(to_drop)} correlated columns to remove.')
print(to_drop)

In [ ]:
# Removendo algumas colunas que tem basicamente o mesmo valor para o modelo e/ou tenha alta correlação
# r4t3, tamviv, tamhog, hhsize = hogar_total
# v18q, mobilephone = v18q1, qmobilephone
# v14a = saniatrio1
# male oposto do female. Retirando female.
# area1 oposto da area2

retira_cols = ['agesq', 'area2', 'hogar_total', 'male', 'public', 'r4t3', 'tamhog', 'tamviv', 
               'hhsize', 'v18q', 'v14a', 'mobilephone', 'female']


parentesco_cols = [colunas for colunas in train.columns.tolist() if 'parentesco' in colunas and 'parentesco1' not in colunas]

retira_cols.extend(parentesco_cols)

train = train.drop(retira_cols, axis=1)
test = test.drop(retira_cols, axis=1)
df_all = df_all.drop(retira_cols, axis=1)

In [ ]:
# Features para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Limitando o treinamento ao chefe da familia

# Criando um novo dataframe para treinar
heads = train[train['parentesco1'] == 1]

In [ ]:
# Criando um novo modelo
rf2 = RandomForestClassifier(n_jobs=-1, n_estimators=200, oob_score=True, random_state=42)

In [ ]:
# Treinando o modelo
rf2.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf2.predict(test[feats]).astype(int)

In [ ]:
test['Target'].value_counts(normalize=True)

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)

# Modelo 3

In [ ]:
# Juntando as abordagens
heads2 = train[train['parentesco1'] == 1]

In [ ]:
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Criando um novo modelo
rf3 = RandomForestClassifier(n_jobs=-1, n_estimators=200, oob_score=True, random_state=42)

In [ ]:
# Treinando o modelo
rf3.fit(heads2[feats], heads2['Target'])

In [ ]:
# Prevendo usando o modelo treinado
test['Target'] = rf3.predict(test[feats]).astype(int)

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
# Feature Importance
pd.Series(rf3.feature_importances_, index=feats).sort_values().plot.barh(figsize = (20, 40));

# Modelo 4 - Melhor

In [ ]:
# Copiando do campeão
rf4 = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=4, n_estimators=700,
                            min_impurity_decrease=1e-3, min_samples_leaf=2,
                            verbose=0, class_weight='balanced')

In [ ]:
# Treinando o modelo
rf4.fit(heads2[feats], heads2['Target'])

In [ ]:
# Prevendo usando o modelo treinado
test['Target'] = rf4.predict(test[feats]).astype(int)

In [ ]:
# Feature Importance
pd.Series(rf4.feature_importances_, index=feats).sort_values().plot.barh(figsize = (20, 40));

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)

Passos para criação da Matriz de Confusão. Código foi verificado no kaggle https://www.kaggle.com/kuriyaman1002/reduce-features-140-85-keeping-f1-score/execution

In [ ]:
# Divisão dos datasets para poder fazer a matriz de confusão.
# Obtensão do f1_score também

import lightgbm as lgb

train_y = train['Target']

X_train, X_test, y_train, y_test = train_test_split(train[feats], train_y, test_size=0.2, random_state=42)

F1_scorer = make_scorer(f1_score, greater_is_better=True, average='macro')

gbm = lgb.LGBMClassifier(boosting_type='dart', objective='multiclassova', class_weight='balanced', random_state=0)

gbm.fit(X_train, y_train)

In [ ]:
import lightgbm as lgb

train_y = train.Target

X_train, X_test, y_train, y_test = train_test_split(train[feats], train_y, test_size=0.2, random_state=42)

F1_scorer = make_scorer(f1_score, greater_is_better=True, average='macro')

gbm = lgb.LGBMClassifier(boosting_type='dart', objective='multiclassova', class_weight='balanced', random_state=0)

gbm.fit(X_train, y_train)

In [ ]:
# Matriz de confusão
y_test_pred = gbm.predict(X_test)
cm = confusion_matrix(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred, average='macro')
print("confusion matrix: \n", cm)
print("macro F1 score: \n", f1)

# Conclusão:

Esse notebook é resultado da matéria de Data Mining da pós graduação em Ciência de Dados do IESB - Brasília.

O notebook do professor Marcos https://www.kaggle.com/marcosvafg/iesb-miner-ii-aula-05-random-forest?scriptVersionId=29607563 foi utilizado como base para esse trabalho.

No decorer da execução do modelo também foram utilizados notebooks submetidos na competição que utilizei aqui também.

O trabalho foi concluído com um valor de 0.44109 para o melhor modelo, o modelo 4. Utilizando RandomForestClassifier com os parâmetros utilizados pelo campeão da competição. Sendo necessário um grande trabalho de verificação das variáveis e de qual modelo seria melhor. Outras validações também foram utilizadas de alguns códigos no kaggle, e estão documentados nesse arquivo.

# Desafios e problemas encontrados

A cada teste realizado o valor diminuia consideravelmente mesmo tomando ações que achava que seria melhor. Enfim, em alguns momentos parece que nada fluiria.

Tentei utilizar o plot_confusion_matrix para melhorar a visualização mas não foi possível. Acredito que a versão no kaggle estava diferente da necessária.